In [10]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportion_confint
golden_label=pd.read_csv("data/total_labeled.csv")['income']
pseudo_label=pd.read_csv("data/total_unlabeled_pred.csv")['income']

In [11]:
golden_success=np.sum(golden_label)
pseudo_success=np.sum(pseudo_label)
n_golden=len(golden_label)
n_pseudo=len(pseudo_label)
print("golden success: ", golden_success)
print("pseudo success: ", pseudo_success)
print("golden size: ", n_golden)
print("pseudo size: ", n_pseudo)

golden success:  2412
pseudo success:  7568
golden size:  10000
pseudo size:  38842


In [13]:
golden_ci_low, golden_ci_high = proportion_confint(count=golden_success, nobs=n_golden, alpha=0.1, method='wilson')
pseudo_ci_low, pseudo_ci_high = proportion_confint(count=pseudo_success, nobs=n_pseudo, alpha=0.1, method='wilson')
print("golden ci: ", f"{golden_ci_low:.3f}", f"{golden_ci_high:.3f}")
print("pseudo ci: ", f"{pseudo_ci_low:.3f}", f"{pseudo_ci_high:.3f}")
print("golden ci width: ", f"{golden_ci_high-golden_ci_low:.3f}")
print("pseudo ci width: ", f"{pseudo_ci_high-pseudo_ci_low:.3f}")


golden ci:  0.234 0.248
pseudo ci:  0.192 0.198
golden ci width:  0.014
pseudo ci width:  0.007
